<a href="https://colab.research.google.com/github/Inglam/Computer-Vision-EECS442-/blob/master/EECS442_PyTorch_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EECS 442 Computer Vision 
# PyTorch Tutorial

2020-10-02

# Introduction

PyTorch will be used throughout the remaining of the semseter. This tutorial first covers the basics of PyTorch. At the end, we will train a neural network for classifying handwritten digits together. Some of the material draws from [EECS498/598](https://web.eecs.umich.edu/~justincj/teaching/eecs498/FA2020/), [DEEP LEARNING WITH PYTORCH: A 60 MINUTE BLITZ](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html#deep-learning-with-pytorch-a-60-minute-blitz), [Python Data Science Handbook](https://learning.oreilly.com/library/view/python-data-science/9781491912126/), and [EECS 504 PyTorch Tutorial](https://drive.google.com/open?id=1lRdeX1rO4KShTM-WbM2qyHXjcur3Iao5).

You can learn more by reading the official PyTorch book written by the PyTorch development team. [link](https://pytorch.org/deep-learning-with-pytorch)


In [ ]:
!pip install torchviz
!pip install torchsummary
import torch
import torchviz
from torchsummary import summary
print(torch.__version__)

     |████████████████████████████████| 51kB 3.3MB/s 
  Created wheel for torchviz: filename=torchviz-0.0.1-cp36-none-any.whl size=3520 sha256=9919ea72a6fb7d3469a0bae3deb704ed7c2d21c855aa4909d445beccb5cb9f12
  Stored in directory: /root/.cache/pip/wheels/2a/c2/c5/b8b4d0f7992c735f6db5bfa3c5f354cf36502037ca2b585667
Successfully built torchviz
1.6.0+cu101


# Tensors
Numpy is a great framework, but it cannot utilize GPUs to accelerate its numerical computations. For modern deep neural networks, GPUs often provide speedups of 50x or greater, so unfortunately numpy won’t be enough for modern deep learning.

Essentially, PyTorch is NumPy that can run on GPUs and allow automatic differentiation for building and training neural networks models. PyTorch tensors are similar to NumPy's ndarrays. 


## Basic Operations

In [ ]:
# Construct a tensor from data
x = torch.tensor([5.5, 3.1])
print(x)

tensor([5.5000, 3.1000])


In [ ]:
# construct a 5x3 matrix, uninitialized
x = torch.empty(5, 3)
print(x)

tensor([[1.1972e-35, 0.0000e+00, 3.3631e-44],
        [0.0000e+00,        nan, 0.0000e+00],
        [1.1578e+27, 1.1362e+30, 7.1547e+22],
        [4.5828e+30, 1.2121e+04, 7.1846e+22],
        [9.2198e-39, 7.0374e+22, 5.0948e-14]])


In [ ]:
# construct a randomly initialized matrix
x = torch.rand(5, 3)
print(x)

tensor([[0.8963, 0.3735, 0.4606],
        [0.6429, 0.5867, 0.9006],
        [0.4683, 0.0620, 0.3558],
        [0.4663, 0.2026, 0.4600],
        [0.2671, 0.8559, 0.3861]])


In [ ]:
# construct a matrix filled with zeros and of dtype long
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [ ]:
# you can get the size of the tensor
print(x.size())

torch.Size([5, 3])


In [ ]:
# similar to numpy.ndarray, torch.tensor supports many basic arithmetic operations
x, y = torch.rand(5, 3), torch.rand(5, 3)
print(x, y, x + y)

# in-place operation
y.add_(x)
print(y)

tensor([[0.8412, 0.2671, 0.7117],
        [0.4817, 0.5758, 0.6747],
        [0.4581, 0.5103, 0.8931],
        [0.0511, 0.4599, 0.6924],
        [0.5441, 0.7341, 0.1286]]) tensor([[0.3077, 0.2417, 0.8462],
        [0.0786, 0.7811, 0.7223],
        [0.7699, 0.6730, 0.3770],
        [0.4401, 0.7300, 0.9534],
        [0.9963, 0.7539, 0.1993]]) tensor([[1.1489, 0.5088, 1.5579],
        [0.5603, 1.3569, 1.3970],
        [1.2280, 1.1834, 1.2701],
        [0.4912, 1.1899, 1.6458],
        [1.5404, 1.4880, 0.3279]])
tensor([[1.1489, 0.5088, 1.5579],
        [0.5603, 1.3569, 1.3970],
        [1.2280, 1.1834, 1.2701],
        [0.4912, 1.1899, 1.6458],
        [1.5404, 1.4880, 0.3279]])


Resizing is a useful operation for torch.tensors. when training machine learning models, somes you want to change shape your input tensor and `torch.view()` method will come in handy.

In [ ]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


To access the Python number sotred in a one element tensor (such as the loss value), you can use .item() to get the value.

In [ ]:
x = torch.randn(1)
print(x, x.item())

tensor([0.8008]) 0.8008484244346619


## Tensor Indexing

Similar to Python lists and numpy arrays, PyTorch tensors can be **sliced** using the syntax `start:stop` or `start:stop:step`. The `stop` index is always non-inclusive: it is the first element not to be included in the slice.

Start and stop indices can be negative, in which case they count backward from the end of the tensor.

In [ ]:
a = torch.tensor([0, 11, 22, 33, 44, 55, 66])
print(0, a)        # (0) Original tensor
print(1, a[2:5])   # (1) Elements between index 2 and 5
print(2, a[2:])    # (2) Elements after index 2
print(3, a[:5])    # (3) Elements before index 5
print(4, a[:])     # (4) All elements
print(5, a[1:5:2]) # (5) Every second element between indices 1 and 5
print(6, a[:-1])   # (6) All but the last element
print(7, a[-4::2]) # (7) Every second element, starting from the fourth-last

0 tensor([ 0, 11, 22, 33, 44, 55, 66])
1 tensor([22, 33, 44])
2 tensor([22, 33, 44, 55, 66])
3 tensor([ 0, 11, 22, 33, 44])
4 tensor([ 0, 11, 22, 33, 44, 55, 66])
5 tensor([11, 33])
6 tensor([ 0, 11, 22, 33, 44, 55])
7 tensor([33, 55])


For multidimensional tensors, you can provide a slice or integer for each dimension of the tensor in order to extract different types of subtensors:

In [ ]:
# Create the following rank 2 tensor with shape (3, 4)
# [[ 1  2  3  4]
#  [ 5  6  7  8]
#  [ 9 10 11 12]]
a = torch.tensor([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
print('Original tensor:')
print(a)
print('shape: ', a.shape)

# Get row 1, and all columns. 
print('\nSingle row:')
print(a[1, :])
print(a[1])  # Gives the same result; we can omit : for trailing dimensions
print('shape: ', a[1].size(), a[1, :].shape)

print('\nSingle column:')
print(a[:, 1])
print('shape: ', a[:, 1].shape)

# Get the first two rows and the last three columns
print('\nFirst two rows, last three columns:')
print(a[:2, -3:])
print('shape: ', a[:2, -3:].shape)

# Get every other row, and columns at index 1 and 2
print('\nEvery other row, middle columns:')
print(a[::2, 1:3])
print('shape: ', a[::2, 1:3].shape)

Original tensor:
tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])
shape:  torch.Size([3, 4])

Single row:
tensor([5, 6, 7, 8])
tensor([5, 6, 7, 8])
shape:  torch.Size([4]) torch.Size([4])

Single column:
tensor([ 2,  6, 10])
shape:  torch.Size([3])

First two rows, last three columns:
tensor([[2, 3, 4],
        [6, 7, 8]])
shape:  torch.Size([2, 3])

Every other row, middle columns:
tensor([[ 2,  3],
        [10, 11]])
shape:  torch.Size([2, 2])


There are two common ways to access a single row or column of a tensor: using an integer will reduce the rank by one, and using a length-one slice will keep the same rank. Note that this is different behavior from MATLAB.

In [ ]:
# Create the following rank 2 tensor with shape (3, 4)
a = torch.tensor([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
print('Original tensor')
print(a)

row_r1 = a[1, :]    # Rank 1 view of the second row of a  
row_r2 = a[1:2, :]  # Rank 2 view of the second row of a
print('\nTwo ways of accessing a single row:')
print(row_r1, row_r1.shape)
print(row_r2, row_r2.shape)

# We can make the same distinction when accessing columns::
col_r1 = a[:, 1]
col_r2 = a[:, 1:2]
print('\nTwo ways of accessing a single column:')
print(col_r1, col_r1.shape)
print(col_r2, col_r2.shape)

Original tensor
tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])

Two ways of accessing a single row:
tensor([5, 6, 7, 8]) torch.Size([4])
tensor([[5, 6, 7, 8]]) torch.Size([1, 4])

Two ways of accessing a single column:
tensor([ 2,  6, 10]) torch.Size([3])
tensor([[ 2],
        [ 6],
        [10]]) torch.Size([3, 1])


Slicing a tensor returns a **view** into the same data, so modifying it will also modify the original tensor. To avoid this, you can use the `clone()` method to make a copy of a tensor.

In [ ]:
# Create a tensor, a slice, and a clone of a slice
a = torch.tensor([[1, 2, 3, 4], [5, 6, 7, 8]])
b = a[0, 1:]
c = a[0, 1:].clone()
print('Before mutating:')
print(a)
print(b)
print(c)

a[0, 1] = 20  # a[0, 1] and b[0] point to the same element
b[1] = 30     # b[1] and a[0, 2] point to the same element
c[2] = 40     # c is a clone, so it has its own data
print('\nAfter mutating:')
print(a)
print(b)
print(c)

print(a.storage().data_ptr() == c.storage().data_ptr())

Before mutating:
tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])
tensor([2, 3, 4])
tensor([2, 3, 4])

After mutating:
tensor([[ 1, 20, 30,  4],
        [ 5,  6,  7,  8]])
tensor([20, 30,  4])
tensor([ 2,  3, 40])
False


Boolean tensor indexing lets you pick out arbitrary elements of a tensor according to a boolean mask. Frequently this type of indexing is used to select or modify the elements of a tensor that satisfy some condition.

In [ ]:
a = torch.tensor([[1,2], [3, 4], [5, 6]])
print('Original tensor:')
print(a)

# Find the elements of a that are bigger than 3. The mask has the same shape as
# a, where each element of mask tells whether the corresponding element of a
# is greater than three.
mask = (a > 3)
print('\nMask tensor:')
print(mask)

# We can use the mask to construct a rank-1 tensor containing the elements of a
# that are selected by the mask
print('\nSelecting elements with the mask:')
print(a[mask])

# We can also use boolean masks to modify tensors; for example this sets all
# elements <= 3 to zero:
a[a <= 3] = 0
print('\nAfter modifying with a mask:')
print(a)

Original tensor:
tensor([[1, 2],
        [3, 4],
        [5, 6]])

Mask tensor:
tensor([[False, False],
        [False,  True],
        [ True,  True]])

Selecting elements with the mask:
tensor([4, 5, 6])

After modifying with a mask:
tensor([[0, 0],
        [0, 4],
        [5, 6]])


## Broadcasting

Broadcasting is a powerful mechanism that allows PyTorch to work with arrays of different shapes when performing arithmetic operations. Frequently we have a smaller tensor and a larger tensor, and we want to use the smaller tensor multiple times to perform some operation on the larger tensor.

For example, suppose that we want to add a constant vector to each row of a tensor. We could do it like this:


In [ ]:
# We will add the vector v to each row of the matrix x,
# storing the result in the matrix y
x = torch.tensor([[1,2,3], [4,5,6], [7,8,9], [10, 11, 12]])
v = torch.tensor([1, 0, 1])
y = torch.zeros_like(x)   # Create an empty matrix with the same shape as x

# Add the vector v to each row of the matrix x with an explicit loop
for i in range(4):
    y[i, :] = x[i, :] + v

print(y)

tensor([[ 2,  2,  4],
        [ 5,  5,  7],
        [ 8,  8, 10],
        [11, 11, 13]])


This works; however when the tensor x is very large, computing an explicit loop in Python could be slow. Note that adding the vector v to each row of the tensor x is equivalent to forming a tensor vv by stacking multiple copies of v vertically, then performing elementwise summation of x and vv. We could implement this approach like this:


In [ ]:
vv = v.repeat((4, 1))  # Stack 4 copies of v on top of each other
print(vv)              # Prints "[[1 0 1]
                       #          [1 0 1]
                       #          [1 0 1]
                       #          [1 0 1]]"

y = x + vv  # Add x and vv elementwise
print(y)

tensor([[1, 0, 1],
        [1, 0, 1],
        [1, 0, 1],
        [1, 0, 1]])
tensor([[ 2,  2,  4],
        [ 5,  5,  7],
        [ 8,  8, 10],
        [11, 11, 13]])


PyTorch broadcasting allows us to perform this computation without actually creating multiple copies of v. Consider this version, using broadcasting:

In [ ]:
# We will add the vector v to each row of the matrix x,
# storing the result in the matrix y
x = torch.tensor([[1,2,3], [4,5,6], [7,8,9], [10, 11, 12]])
v = torch.tensor([1, 0, 1])
y = x + v  # Add v to each row of x using broadcasting
print(y)

tensor([[ 2,  2,  4],
        [ 5,  5,  7],
        [ 8,  8, 10],
        [11, 11, 13]])


The line y = x + v works even though x has shape (4, 3) and v has shape (3,) due to broadcasting; this line works as if v actually had shape (4, 3), where each row was a copy of v, and the sum was performed elementwise.

Broadcasting two tensors together follows these rules:

1.   If the tensors do not have the same rank, prepend the shape of the lower rank array with 1s until both shapes have the same length.
2.   The two tensors are said to be *compatible* in a dimension if they have the same size in the dimension, or if one of the tensors has size 1 in that dimension.
3.   The tensors can be broadcast together if they are compatible in all dimensions.
4.   After broadcasting, each tensor behaves as if it had shape equal to the elementwise maximum of shapes of the two input tensors.
5.   In any dimension where one tensor had size 1 and the other tensor had size greater than 1, the first tensor behaves as if it were copied along that dimension

If this explanation does not make sense, try reading the explanation from the [documentation](https://pytorch.org/docs/stable/notes/broadcasting.html).

Not all functions support broadcasting. You can find functions that does not support broadcasting from the official docs. (e.g. [`torch.mm`](https://pytorch.org/docs/stable/torch.html#torch.mm) does not support broadcasting, but [`torch.matmul`](https://pytorch.org/docs/1.1.0/torch.html#torch.matmul) does)

Broadcasting can let us easily implement many different operations. For example we can compute an outer product of vectors:

In [ ]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]])  # x has shape (2, 3)
c = torch.tensor([1, 10, 11, 100])        # c has shape (4)
print('Here is the matrix:')
print(x)
print('\nHere is the vector:')
print(c)

# We do the following:
# 1. Reshape c from (4,) to (4, 1, 1)
# 2. x has shape (2, 3). Since they have different ranks, when we multiply the
#    two, x behaves as if its shape were (1, 2, 3)
# 3. The result of the broadcast multiplication between tensor of shape
#    (4, 1, 1) and (1, 2, 3) has shape (4, 2, 3)
# 4. The result y has shape (4, 2, 3), and y[i] (shape (2, 3)) is equal to
#    c[i] * x
y = c.view(-1, 1, 1) * x
print('\nMultiply x by a set of constants:')
print(y)

Here is the matrix:
tensor([[1, 2, 3],
        [4, 5, 6]])

Here is the vector:
tensor([  1,  10,  11, 100])

Multiply x by a set of constants:
tensor([[[  1,   2,   3],
         [  4,   5,   6]],

        [[ 10,  20,  30],
         [ 40,  50,  60]],

        [[ 11,  22,  33],
         [ 44,  55,  66]],

        [[100, 200, 300],
         [400, 500, 600]]])


## NumPy Bridge

We can easily convert a torch.tensor to a numpy.ndarray and vice versa.

In [ ]:
# torch.tensor to np.array
a = torch.ones(5)
b = a.numpy()
print(a, b)

# np.array to torch.tensor
c = torch.from_numpy(b)
print(b, c)

tensor([1., 1., 1., 1., 1.]) [1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1.] tensor([1., 1., 1., 1., 1.])


## CUDA Tensors

So far, all tensors we used are on the CPU. With the `.to()` method, we can move the tensor to the GPU.

Matrix operations on GPUs are much faster than on CPUs.

In [ ]:
device = torch.device('cuda')
x  = torch.rand(10)
print(x)
x = x.to(device)
print(x)

tensor([0.6705, 0.7627, 0.9312, 0.1194, 0.9688, 0.9666, 0.3719, 0.4897, 0.9837,
        0.1093])


In [ ]:
import numpy as np
x, y = np.random.rand(8000, 1000), np.random.rand(1000, 8000)
x_cuda, y_cuda = torch.from_numpy(x).to('cuda'), torch.from_numpy(y).to('cuda')

In [ ]:
%%time
z = np.dot(x, y)

In [ ]:
%%time
z = torch.matmul(x_cuda, y_cuda)

By oprating on GPUs, we achieved ~1000X speedup.

## Automatic Differentiation

Automatic differentiation the central functionality of PyTorch. The `autograd` package provides automatic differentiation for all operations on Tensors. 

``torch.Tensor`` is the central class of the package. If you set its attribute
``.requires_grad`` as ``True``, it starts to track all operations on it. When
you finish your computation you can call ``.backward()`` and have all the
gradients computed automatically. The gradient for this tensor will be
accumulated into ``.grad`` attribute.

To stop a tensor from tracking history, you can call ``.detach()`` to detach
it from the computation history, and to prevent future computation from being
tracked.

To prevent tracking history (and using memory), you can also wrap the code block
in ``with torch.no_grad():``. This can be particularly helpful when evaluating a
model because the model may have trainable parameters with
``requires_grad=True``, but for which we don't need the gradients.

In [ ]:
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
print(y.grad_fn)

The computation graph looks like below. 

In [ ]:
torchviz.make_dot(y)

Now let's try to do more operations on ``y`` and see how the graph would change.

In [ ]:
z = y * y * 3
out = z.mean()
torchviz.make_dot(out)

As you can see, PyTorch can automatically track our operations on ``torch.tensor``.

Now, let's take a look at how to compute the gradients.

$$o = \frac{1}{4}\sum_{i=1}^4 3(x_i + 2)^2$$

In [ ]:
out.backward()
print(x.grad)

``.backward()`` will automatically detele the stored computation graph. Calling `.backward()` on the same tensor for the second time will cause an error.

In [ ]:
out.backward()

You can set `retain_graph=True` when calling `.backward()` so that the graph won't be deleted.

In [ ]:
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
z = y * y * 3
out = z.mean()
out.backward(retain_graph=True)
# call the second time
out.backward()
print(x.grad)

 Notice that now the elements value in `x.grad` has been changed to `9`. This is bacause PyTorch will cumulatively sum up all backward gradients.

You can also stop autograd from tracking history on Tensors
with ``.requires_grad=True`` either by wrapping the code block in
``with torch.no_grad():``

In [ ]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

Or by using ``.detach()`` to get a new Tensor with the same
content but that does not require gradients:


In [ ]:
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())

# Debugging in Colab

Most of the time when a Python script fails, it will raise an Exception.
When the interpreter hits one of these exceptions, information about the cause of the error can be found in the *traceback*, which can be accessed from within Python.
With the ``%xmode`` magic function, IPython allows you to control the amount of information printed when the exception is raised.
Consider the following code:

In [ ]:
def func1(a, b):
    return a / b

def func2(x):
    a = x
    b = x - 1
    return func1(a, b)

In [ ]:
func2(1)

In [ ]:
# most compact
%xmode Plain
func2(1)

In [ ]:
# default
%xmode Context
func2(1)

In [ ]:
# gives additional information about function argument
%xmode Verbose
func2(1)

## ipdb
The standard Python tool for interactive debugging is ``pdb``, the Python debugger.
This debugger lets the user step through the code line by line in order to see what might be causing a more difficult error.
The IPython-enhanced version of this is ``ipdb``, the IPython debugger.

There are many ways to launch and use both these debuggers; we won't cover them fully here.
Refer to the online documentation of these two utilities to learn more.

In IPython, perhaps the most convenient interface to debugging is the ``%debug`` magic command.
If you call it after hitting an exception, it will automatically open an interactive debugging prompt at the point of the exception.
The ``ipdb`` prompt lets you explore the current state of the stack, explore the available variables, and even run Python commands!

Let's look at the most recent exception, then do some basic tasks–print the values of ``a`` and ``b``, and type ``quit`` to quit the debugging session:

In [ ]:
func2(1)

In [ ]:
%debug

**Remember: after you are done with debugging, please remove the extra lines/blocks of code before submsision.**

## Partial list of debugging commands

There are many more available commands for interactive debugging than we've listed here; the following table contains a description of some of the more common and useful ones:

| Command         |  Description                                                |
|-----------------|-------------------------------------------------------------|
| ``list``        | Show the current location in the file                       |
| ``h(elp)``      | Show a list of commands, or find help on a specific command |
| ``q(uit)``      | Quit the debugger and the program                           |
| ``c(ontinue)``  | Quit the debugger, continue in the program                  |
| ``n(ext)``      | Go to the next step of the program                          |
| ``<enter>``     | Repeat the previous command                                 |
| ``p(rint)``     | Print variables                                             |
| ``s(tep)``      | Step into a subroutine                                      |
| ``r(eturn)``    | Return out of a subroutine                                  |

For more information, use the ``help`` command in the debugger, or take a look at ``ipdb``'s [online documentation](https://github.com/gotcha/ipdb).

# Build and Train a Neural Network 

`torch.nn` is the package that we are going to use a lot in EECS 442. It provides many useful built-in functions for building neural networks. 

First, let's take a look at how to define a neural network.

For any neural network built in PyTorch, user will need to specific `__init__()` and `forward()`.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.out = nn.Linear(64, 10)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        y = self.out(h)
        return y

net = Net()
print(net)

We count the number of parameters in the neural network.

In [ ]:
def count_parameters(model):
    return sum(layer.numel() for layer in model.parameters() if layer.requires_grad)

In [ ]:
print(f'Total parameters: {count_parameters(net)}')

Now, let's try a random input and compute the loss. We will define a mean sqaured error as the loss function.

In [ ]:
input = torch.rand(1, 784)
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)

criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

Great! We have successfully calculated the loss value on a pair of random input-target. Now, let's try to do backpropagation to update the neural network so that it can output approximately the groundtruth target.

Remember that `.backward()` would accumulate the gradients. It's a good practice to zero the gradient buffer before we do any backpropagation.

In [ ]:
net.zero_grad()
loss.backward()
print(list(net.parameters())[0].grad)

The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):
``weight = weight - learning_rate * gradient``.

In [ ]:
import matplotlib.pyplot as plt
from tqdm.notebook import tnrange # progress bar
import numpy as np

In [ ]:
net = Net()
num_iters = 1000
learning_rate = 0.01
loss_traj = []

for _ in tnrange(num_iters):
    # step 1: compute loss
    output = net(input)
    loss = criterion(output, target)

    # step 2: compute gradient
    net.zero_grad()
    loss.backward()

    # step 3: update the model parameters
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

    # keep track the loss values for visualization
    loss_traj.append(loss.item())

In [ ]:
plt.plot(np.arange(num_iters), loss_traj)

Great! We have successfully trained our first neural network to perform perfectly on a random data.

# Train a Classifier

Train a neural network on a single random data point is not very exciting. Now, let's try to put together what we have learned and train a 2-layer neural network to classify the images in CIFAR-10 dataset.

We can easily download the datafrom from `torchvision.datasets`. `torch.utils.data.DataLoader` is useful for creating a data feeding pipeline.

In [ ]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=256,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images[:4]))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

Previously, we trained a simple multilayer perceptron. Here, we will define and train a convolutional neural network.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = x.view(-1, 32 * 32 * 3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# initialize the neural netowrk and send it to the GPU
net = Net().cuda()
summary(net, input_size = (3, 32, 32))

Now, we have successfully defined a neural network with 2 convolutional layers and 3 linear layers. Let's define the loss, the optimizer, and train the neural network.

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 5e-4)

num_epochs = 10

for epoch in tnrange(num_epochs):

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f'Epoch {epoch} loss: {running_loss}')
    running_loss = 0.0

Save the trained model parameters.

In [ ]:
# save the trained model
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

# load the saved model
net = Net()
net.load_state_dict(torch.load(PATH))

In [ ]:
correct = 0
total = 0
pred_list = []
labels_list = []
# no need to keep track of the computation graph when testing
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        labels_list.append(labels.detach().numpy())
        pred_list.append(predicted.detach().numpy())

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Now, let's plot the confusion matrix to understand how the model performs on each of the class.

In [ ]:
labels = np.concatenate(labels_list)
preds = np.concatenate(pred_list)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
conf_matrix = confusion_matrix(preds, labels)

In [ ]:
sns.heatmap(conf_matrix, cmap="YlGnBu", yticklabels=classes, xticklabels=classes)

The network tends to misclassify cat as dog, truck as car, deer as bird.

# Excercise (Optional)

Now you have a good understanding of the basics of PyTorch. Now it's your turn to practice your new skills.


## Basic Operations

Use [`torch.bmm`](https://pytorch.org/docs/1.1.0/torch.html#torch.bmm) to perform a batched matrix multiply.


In [ ]:
B, N, M, P = 3, 2, 5, 4
x = torch.rand(B, N, M)  # Random tensor of shape (B, N, M)
y = torch.rand(B, M, P)  # Random tensor of shape (B, M, P)

# We can use a for loop to (inefficiently) compute a batch of matrix multiply
# operations
z1 = torch.empty(B, N, P)  # Empty tensor of shape (B, N, P)
for i in range(B):
  z1[i] = x[i].mm(y[i])
print('Here is the result of batched matrix multiply with a loop:')
print(z1)

z2 = None
##############################################################################
# TODO: Use bmm to compute a batched matrix multiply between x and y; store  #
# the result in z2.                                                          #
##############################################################################
# Replace "pass" statement with your code
pass
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################
print('\nHere is the result of batched matrix multiply with bmm:')
print(z2)

# The two may not return exactly the same result; different linear algebra
# routines often return slightly different results due to the fact that
# floating-point math is non-exact and non-associative.
diff = (z1 - z2).abs().max().item()
print('\nDifference:', diff)
print('Difference within threshold:', diff < 1e-6)

## Tensor Indexing

Practice with boolean masks by implementing the following function:

In [ ]:
def num_negative(x):
  """
  Return the number of negative values in the tensor x
 
  Inputs:
  - x: A tensor of any shape
 
  Returns:
  - num_neg: Number of negative values in x
  """
  num_neg = 0
  ##############################################################################
  # TODO: Use boolean masks to count the number of negative elements in x.     #
  ##############################################################################
  # Replace "pass" statement with your code
  pass
  ##############################################################################
  #                             END OF YOUR CODE                               #
  ##############################################################################
  return num_neg

# Make a few test cases
torch.manual_seed(598)
x0 = torch.tensor([[-1, -1, 0], [0, 1, 2], [3, 4, 5]])
x1 = torch.tensor([0, 1, 2, 3])
x2 = torch.randn(100, 100)
assert num_negative(x0) == 2
assert num_negative(x1) == 0
assert num_negative(x2) == 4984
print('num_negative seems to be correct!')

##  Broadcasting

Write a function that normalizes the columns of a matrix. It should compute the mean and standard deviation of each column, then subtract the mean and divide by the standard deviation for each element in the column.

Example:
```
x = [[ 0,  30,  600],
     [ 1,  10,  200],
     [-1,  20,  400]]
```
- The first column has mean 0 and std 1
- The second column has mean 20 and std 10
- The third column has mean 400 and std 200

After normalizing the columns, the result should be:
```
y = [[ 0,  1,  1],
     [ 1, -1, -1],
     [-1,  0,  0]]
```

In [ ]:
def normalize_columns(x):
  """
  Normalize the columns of a matrix by subtracting the mean and dividing by the
  standard deviation.
 
  Inputs:
  - x: Tensor of shape (N, M)
  
  Returns:
  - y: Tensor of shape (N, M) which is a copy of x with normalized columns.
  """
  y = x.clone()
  ##############################################################################
  # TODO: Complete the implementation of this function. Do not modify x.       #
  # Your implementation should not use any loops; instead you should use       #
  # reduction and broadcasting operations.                                     #
  ##############################################################################
  # Replace "pass" statement with your code
  pass
  ##############################################################################
  #                             END OF YOUR CODE                               #
  ##############################################################################
  return y

In [ ]:
x0 = torch.tensor([[0., 30., 600.], [1., 10., 200.], [-1., 20., 400.]])
y0 = normalize_columns(x0)
print('Here is x0:')
print(x0)
print('Here is y0:')
print(y0)
assert y0.tolist() == [[0., 1., 1.], [1., -1., -1.], [-1., 0., 0.]]
assert x0.tolist() == [[0., 30., 600.], [1., 10., 200.], [-1., 20., 400.]]